# Aprendizaje para la clasificación de dibujos en Pictionary

En este proyecto, vamos a simular el proceso de aprendizaje del software del juego Pictionary con una red neuronal que utilizará una capa LSTM para las secuencias de trazos y luego una capa densa para determinar la categoría a la que pertenece un dibujo en concreto.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import OneHotEncoder

En primer lugar, cargaremos en un vector las distintas categorías posibles.

In [2]:
categories = open('categories.txt','r').read().splitlines()
np.random.shuffle(categories)

Definimos una cantidad concreta de categorías para usar en el aprendizaje y un máximo de dibujos para cada categoría.
A continuación, iremos agregando los dibujos correspondientes a los conjuntos de entrenamiento, validación y test. En cuanto a las categorías, serán almacenadas como números enteros desde 0 hasta max_cat-1 en formato one-hot.

In [3]:
max_cat = 8
max_pic = 500
x_train = []
x_val = []
x_test = []
trainY = []
valY = []
testY = []
i = 1
for c in categories[:max_cat]:
    npz_file = np.load('all_sketches/' + c + '.npz', encoding='latin1', allow_pickle=True)
    train_set = np.array(npz_file['train'])[:max_pic]
    val_set = np.array(npz_file['valid'])[:max_pic]
    test_set = np.array(npz_file['test'])[:max_pic]
    x_train.append(train_set)
    x_val.append(val_set)
    x_test.append(test_set)
    for k in range(0,max_pic):
        trainY.append([i-1])
        valY.append([i-1])
        testY.append([i-1])
    print("Categoría", c, ", agregados ", i, " de ", max_cat)
    i = i + 1
        
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)
trainY = np.array(trainY)
valY = np.array(valY)
testY = np.array(testY)

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(trainY)

trainY = one_hot_encoder.transform(trainY)
valY = one_hot_encoder.transform(valY)
testY = one_hot_encoder.transform(testY)

Categoría hat , agregados  1  de  8
Categoría jail , agregados  2  de  8
Categoría door , agregados  3  de  8
Categoría drill , agregados  4  de  8
Categoría traffic light , agregados  5  de  8
Categoría teddy-bear , agregados  6  de  8
Categoría tent , agregados  7  de  8
Categoría mountain , agregados  8  de  8


El siguiente paso será determinar cuál es la cantidad máxima de trazos posibles entre todos los dibujos obtenidos.

In [4]:
max_size = 0
for i in range(0,max_cat):
    for j in range(0,x_train.shape[1]):
        if (x_train[i][j].shape[0] > max_size):
            max_size = x_train[i][j].shape[0]
for i in range(0,max_cat):
    for j in range(0,x_val.shape[1]):
        if (x_val[i][j].shape[0] > max_size):
            max_size = x_val[i][j].shape[0]
for i in range(0,max_cat):
    for j in range(0,x_test.shape[1]):
        if (x_test[i][j].shape[0] > max_size):
            max_size = x_test[i][j].shape[0]
print(max_size)

156


A continuación, redimensionamos los dibujos de los tres conjuntos de datos de forma que todos tienen el mmismo número de trazos (max_size), rellenando con ceros los trazos añadidos, indicando que no se desplazaría más el lápiz sobre el lienzo.

In [6]:
trainX = np.zeros((max_cat*x_train.shape[1],max_size,3))

for i in range(0,max_cat):
    for j in range(0,x_train.shape[1]):
        current_size = x_train[i][j].shape[0]
        for k in range(0,max_size):
            if (k < current_size):
                trainX[x_train.shape[1]*i+j][k] = x_train[i][j][k]

In [7]:
valX = np.zeros((max_cat*x_val.shape[1],max_size,3))

for i in range(0,max_cat):
    for j in range(0,x_val.shape[1]):
        current_size = x_val[i][j].shape[0]
        for k in range(0,max_size):
            if (k < current_size):
                valX[x_val.shape[1]*i+j][k] = x_val[i][j][k]

In [8]:
testX = np.zeros((max_cat*x_test.shape[1],max_size,3))

for i in range(0,max_cat):
    for j in range(0,x_test.shape[1]):
        current_size = x_test[i][j].shape[0]
        for k in range(0,max_size):
            if (k < current_size):
                testX[x_test.shape[1]*i+j][k] = x_test[i][j][k]

Ahora que los datos han sido tratados, procedemos a entrenar nuestra red neuronal. El número de épocas que estableceremos será de 70 con un tamaño de batch igual a 4. Para el aprendizaje, utilizaremos el optimizador ADAM y la función de pérdida será el error cuadrático medio.

In [9]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(max_size))
model.add(Dense(max_cat))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=70, batch_size=4, verbose=1)

Train on 4000 samples
Epoch 1/70
4000/4000 [==============================] - 87s 22ms/sample - loss: 0.1034
Epoch 2/70
4000/4000 [==============================] - 88s 22ms/sample - loss: 0.0928
Epoch 3/70
4000/4000 [==============================] - 87s 22ms/sample - loss: 0.0861
Epoch 4/70
4000/4000 [==============================] - 86s 21ms/sample - loss: 0.0817
Epoch 5/70
4000/4000 [==============================] - 88s 22ms/sample - loss: 0.0684
Epoch 6/70
4000/4000 [==============================] - 87s 22ms/sample - loss: 0.0589
Epoch 7/70
4000/4000 [==============================] - 88s 22ms/sample - loss: 0.0528
Epoch 8/70
4000/4000 [==============================] - 88s 22ms/sample - loss: 0.0471
Epoch 9/70
4000/4000 [==============================] - 91s 23ms/sample - loss: 0.0417
Epoch 10/70
4000/4000 [==============================] - 91s 23ms/sample - loss: 0.0368
Epoch 11/70
4000/4000 [==============================] - 89s 22ms/sample - loss: 0.0330
Epoch 12/70
4000/40

Una vez realizado el entrenamiento de la red, procedemos a predecir, para el conjunto de test, las categorías a las que pertenecen los dibujos del mismo.

In [10]:
testPredict = model.predict(testX)

In [11]:
testY

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [12]:
testPredict

array([[ 1.01026595e+00,  3.02074552e-02,  5.25811948e-02, ...,
        -4.50789742e-03,  3.77699733e-03, -2.11828277e-02],
       [ 1.04170942e+00,  4.80854511e-03, -1.29290335e-02, ...,
         1.48105063e-03, -1.51795894e-03, -1.08353347e-02],
       [ 1.02358580e+00,  9.19699669e-03, -1.80695727e-02, ...,
        -1.83173828e-03,  3.00072134e-04, -8.39058310e-03],
       ...,
       [-1.42145660e-02, -2.26341188e-03,  2.09575817e-02, ...,
         4.85654734e-03,  9.48014855e-03,  9.96936262e-01],
       [-1.47412177e-02, -1.73132867e-03,  2.07811221e-02, ...,
         4.80932556e-03,  8.50234181e-03,  9.99132752e-01],
       [-1.47200543e-02, -1.73144042e-03,  2.07746252e-02, ...,
         4.80678491e-03,  8.49179178e-03,  9.99131322e-01]], dtype=float32)

Podemos determinar cuál es el error cometido en cada categoría, contando para cada una el númmero de errores en función del número de dibujos de cada categoría.

In [14]:
for i in range(0,max_cat):
    total_error = 0
    for j in range(i*max_pic, (i+1)*max_pic):
        if (np.argmax(testPredict[j]) != np.argmax(testY[j])): total_error += 1
    print("Error en la categoría", categories[i], ": ", (total_error/(len(testY)/max_cat))*100, " %")

Error en la categoría hat :  11.799999999999999  %
Error en la categoría jail :  6.0  %
Error en la categoría door :  11.4  %
Error en la categoría drill :  25.6  %
Error en la categoría traffic light :  7.6  %
Error en la categoría teddy-bear :  2.6  %
Error en la categoría tent :  17.4  %
Error en la categoría mountain :  6.4  %


Como podemos comprobar, la red comete más errores al intentar predecir etiquetas de dibujos como taladros, meintras que otros más sencillos de dibujar como las montañas son más fáciles de predecir.